In [102]:
import pandas as pd
import numpy as np
import pyreadstat
import matplotlib.pyplot as plt
import seaborn as sns

import re

In [45]:
df1 = pd.read_csv('../data/crawling_movie_info_modified.csv')
df2 = pd.read_csv('../data/crawling_netizen_review_modified.csv')
df3 = pd.read_csv('../data/korea_netflix_modified.csv')
df4 = pd.read_csv('../data/netflix-rotten-tomatoes-metacritic-imdb.csv')

# this csv file created in this ipynb
# this csv contains top 200 netflix movie data sorted by number of netizend's review
df5 = pd.read_csv('../data/favorite_contents_netflix_movie.csv')

In [46]:
movie_df = df1.copy()
netizen_df = df2.copy()
netflix_df = df3.copy()
total_df = df4.copy()

favorite_top200_df = df5.copy()

In [4]:
netizen_df

,code,korean_title,num_of_netizen,key_review,score_male,score_female,scores_per_age
0,203096,걸스 오브 막시,140,이 영화는 20대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,6.12,9.86,"[9.76, 9.83, 8.92, 9.38, 10.0]"
1,17997,첨밀밀,"1,851",이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,9.19,9.13,"[8.86, 9.05, 9.27, 9.48, 9.35]"
2,181925,클로젯,"6,242",이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,5.55,7.04,"[7.66, 6.11, 5.95, 6.11, 6.33]"
3,197517,마리오네트,121,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,7.42,8.25,"[10.0, 7.89, 8.28, 7.52, 6.6]"
4,155356,미쓰백,"8,671",이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,8.29,9.60,"[9.69, 9.36, 8.7, 8.55, 8.51]"
...,...,...,...,...,...,...,...
1202,72043,쓰리 데이즈,"1,627",이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,8.39,8.03,"[7.65, 8.41, 8.51, 8.34, 8.26]"
1203,77566,브레이킹 던 part2,"7,053",이 영화는 10대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,7.95,9.20,"[9.22, 8.59, 7.95, 8.76, 8.08]"
1204,38227,첫 키스만 50번째,"3,682",이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,9.26,9.24,"[9.41, 9.17, 9.15, 9.22, 8.98]"
1205,74567,브레이킹 던 part1,"3,290",이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.,5.41,7.97,"[7.71, 6.25, 5.33, 6.46, 9.1]"


In [5]:
movie_df.head(1)

,code,korean_title,english_title,genre_list,country,released_date,director,actors
0,203096,걸스 오브 막시,moxie,['드라마'],['미국'],NaN,에이미 포엘러,"['조세핀 랭포드', ' 패트릭 슈왈제네거더보기']"


In [6]:
movie_netizen_merged_df = pd.merge(movie_df[['code', 'korean_title', 'english_title']], netizen_df[['code', 'num_of_netizen']], how='inner', on='code')

###  Exclude comma from num_of_netizen

In [7]:
def exclude_comma(str_):
    str_ = str_.replace(',', '')
    return str_


In [8]:
movie_netizen_merged_df['num_of_netizen'] = movie_netizen_merged_df['num_of_netizen'].apply(exclude_comma).astype(int)

In [9]:
print(movie_netizen_merged_df.iloc[0,-1], type(movie_netizen_merged_df.iloc[0,-1]))
movie_netizen_merged_df.head()


140 <class 'numpy.int32'>


,code,korean_title,english_title,num_of_netizen
0,203096,걸스 오브 막시,moxie,140
1,17997,첨밀밀,comradesalmostalovestory,1851
2,181925,클로젯,thecloset,6242
3,197517,마리오네트,marionette,121
4,155356,미쓰백,missbaek,8671


In [10]:
movie_netizen_merged_sorted_df = movie_netizen_merged_df.sort_values(by=['num_of_netizen'], ascending=False)
movie_netizen_merged_sorted_df

,code,korean_title,english_title,num_of_netizen
729,130966,부산행,traintobusan,59260
567,85579,신과함께-죄와 벌,alongwiththegodsthetwoworlds,57853
288,146506,군함도,thebattleshipisland,53570
667,47384,26년,26years,51593
899,45290,인터스텔라,interstellar,51161
...,...,...,...,...
964,195965,라스트 솔져스,thirst,1
649,167650,엘 카미노 크리스마스,elcaminochristmas,1
58,183839,태극권,taichiwarriors,1
258,187828,쇼 브라더스의 쿵푸 신드롬,ironfistsandkungfukicks,1


In [11]:
movie_netizen_merged_sorted_df[movie_netizen_merged_sorted_df['english_title']=='bewithyou']

,code,korean_title,english_title,num_of_netizen
319,168298,지금 만나러 갑니다,bewithyou,13854


In [12]:
top200_movie_df = movie_netizen_merged_sorted_df.head(201)
top200_movie_df


,code,korean_title,english_title,num_of_netizen
729,130966,부산행,traintobusan,59260
567,85579,신과함께-죄와 벌,alongwiththegodsthetwoworlds,57853
288,146506,군함도,thebattleshipisland,53570
667,47384,26년,26years,51593
899,45290,인터스텔라,interstellar,51161
...,...,...,...,...
865,120123,간신,thetreacherous,6394
318,58333,혐오스런 마츠코의 일생,memoriesofmatsuko,6381
627,40133,다이 하드 4.0,diehard40,6378
1050,84189,돈의 맛,thetasteofmoney,6350


## Merge netflix_df and top200_movie_df with left join

In [13]:
netflix_df.head()

,Unnamed: 0,Title,Release Date,Series or Movies,drama,comedy,action,thriller,romance,crime,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,Title for Crawling,Image
0,4,Moxie,2011-09-22,Movie,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,moxie,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...
1,27,Comrades Almost a Love Story,1996-11-02,Movie,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,comradesalmostalovestory,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...
2,28,The Closet,2001-08-10,Movie,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,thecloset,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...
3,53,The Bold Type,2017-06-20,Series,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,theboldtype,https://occ-0-768-769.1.nflxso.net/dnm/api/v6/...
4,55,Lobster Cop,2018-06-22,Movie,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,lobstercop,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...


In [14]:
movie_netizen_merged_df = pd.merge(movie_df[['code', 'korean_title', 'english_title']], netizen_df[['code', 'num_of_netizen']], how='inner', on='code')

In [15]:
netflix_top200_merged_df = pd.merge(netflix_df, top200_movie_df[['korean_title', 'english_title', 'num_of_netizen']], how='left', left_on='Title for Crawling', right_on='english_title')

In [16]:
temp_df = netflix_top200_merged_df[~netflix_top200_merged_df['num_of_netizen'].isna()]
temp_df

,Unnamed: 0,Title,Release Date,Series or Movies,drama,comedy,action,thriller,romance,crime,...,musical,reality-tv,western,game-show,talk-show,Title for Crawling,Image,korean_title,english_title,num_of_netizen
7,58,Miss Baek,2018-10-11,Movie,1,0,0,0,0,0,...,0,0,0,0,0,missbaek,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,미쓰백,missbaek,8671.0
23,74,Keys To The Heart,2018-01-26,Movie,1,1,0,0,0,0,...,0,0,0,0,0,keystotheheart,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,그것만이 내 세상,keystotheheart,15829.0
24,75,Intimate Strangers,2004-09-03,Movie,1,0,0,1,1,0,...,0,0,0,0,0,intimatestrangers,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,완벽한 타인,intimatestrangers,23874.0
27,78,Gonjiam Haunted Asylum,2018-03-28,Movie,0,0,0,0,0,0,...,0,0,0,0,0,gonjiamhauntedasylum,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,곤지암,gonjiamhauntedasylum,16845.0
28,79,Golden Slumber,2010-01-30,Movie,1,0,1,1,0,0,...,0,0,0,0,0,goldenslumber,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,골든슬럼버,goldenslumber,9609.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,14747,The Bourne Ultimatum,2007-08-03,Movie,0,0,1,1,0,0,...,0,0,0,0,0,thebourneultimatum,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,본 얼티메이텀,thebourneultimatum,8715.0
4738,15141,Pans Labyrinth,2007-01-19,Movie,1,0,0,0,0,0,...,0,0,0,0,0,panslabyrinth,https://occ-0-2042-299.1.nflxso.net/dnm/api/v6...,판의 미로 - 오필리아와 세 개의 열쇠,panslabyrinth,7478.0
4756,15214,The Twilight Saga New Moon,2009-11-20,Movie,1,0,0,0,1,0,...,0,0,0,0,0,thetwilightsaganewmoon,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,뉴 문,thetwilightsaganewmoon,9588.0
4800,15358,Battle Los Angeles,2011-03-11,Movie,0,0,1,0,0,0,...,0,0,0,0,0,battlelosangeles,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,월드 인베이젼,battlelosangeles,7410.0


In [17]:
temp_df[temp_df.duplicated(subset=['Title'], keep=False)]

,Unnamed: 0,Title,Release Date,Series or Movies,drama,comedy,action,thriller,romance,crime,...,musical,reality-tv,western,game-show,talk-show,Title for Crawling,Image,korean_title,english_title,num_of_netizen
293,1278,Start-Up,2019-12-18,Series,1,1,1,0,0,0,...,0,0,0,0,0,startup,https://occ-0-1091-299.1.nflxso.net/dnm/api/v6...,시동,startup,10726.0
501,1948,About Time,2013-11-08,Series,1,1,0,0,1,0,...,0,0,0,0,0,abouttime,https://occ-0-58-2773.1.nflxso.net/dnm/api/v6/...,어바웃 타임,abouttime,15009.0
768,3258,Snowpiercer,2014-07-11,Series,1,0,1,0,0,0,...,0,0,0,0,0,snowpiercer,https://occ-0-1091-299.1.nflxso.net/dnm/api/v6...,설국열차,snowpiercer,37844.0
901,3708,Start-Up,2019-12-18,Movie,1,1,1,0,0,0,...,0,0,0,0,0,startup,https://occ-0-1007-1360.1.nflxso.net/dnm/api/v...,시동,startup,10726.0
1427,5231,Be with You,2018-04-06,Movie,1,0,0,0,1,0,...,0,0,0,0,0,bewithyou,https://occ-0-1007-1360.1.nflxso.net/dnm/api/v...,지금 만나러 갑니다,bewithyou,13854.0
3657,10933,Be with You,2015-10-31,Series,0,0,0,0,0,0,...,0,0,0,0,0,bewithyou,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,지금 만나러 갑니다,bewithyou,13854.0
4040,12142,Be with You,2004-10-30,Movie,1,0,0,0,1,0,...,0,0,0,0,0,bewithyou,https://occ-0-1007-1360.1.nflxso.net/dnm/api/v...,지금 만나러 갑니다,bewithyou,13854.0
4106,12410,About Time,2013-11-08,Movie,1,1,0,0,1,0,...,0,0,0,0,0,abouttime,https://occ-0-2219-2218.1.nflxso.net/dnm/api/v...,어바웃 타임,abouttime,15009.0
4450,13920,Snowpiercer,2014-07-11,Movie,1,0,1,0,0,0,...,0,0,0,0,0,snowpiercer,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,설국열차,snowpiercer,37844.0


In [18]:
temp_df2 = temp_df.copy()

In [19]:
to_rename = {'Series or Movies':'series_or_movies', 'Release Date':'release_date', 'Title':'title'}

In [20]:
temp_df2 = temp_df2.rename(columns=to_rename)
temp_df2

,Unnamed: 0,title,release_date,series_or_movies,drama,comedy,action,thriller,romance,crime,...,musical,reality-tv,western,game-show,talk-show,Title for Crawling,Image,korean_title,english_title,num_of_netizen
7,58,Miss Baek,2018-10-11,Movie,1,0,0,0,0,0,...,0,0,0,0,0,missbaek,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,미쓰백,missbaek,8671.0
23,74,Keys To The Heart,2018-01-26,Movie,1,1,0,0,0,0,...,0,0,0,0,0,keystotheheart,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,그것만이 내 세상,keystotheheart,15829.0
24,75,Intimate Strangers,2004-09-03,Movie,1,0,0,1,1,0,...,0,0,0,0,0,intimatestrangers,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,완벽한 타인,intimatestrangers,23874.0
27,78,Gonjiam Haunted Asylum,2018-03-28,Movie,0,0,0,0,0,0,...,0,0,0,0,0,gonjiamhauntedasylum,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,곤지암,gonjiamhauntedasylum,16845.0
28,79,Golden Slumber,2010-01-30,Movie,1,0,1,1,0,0,...,0,0,0,0,0,goldenslumber,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,골든슬럼버,goldenslumber,9609.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,14747,The Bourne Ultimatum,2007-08-03,Movie,0,0,1,1,0,0,...,0,0,0,0,0,thebourneultimatum,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,본 얼티메이텀,thebourneultimatum,8715.0
4738,15141,Pans Labyrinth,2007-01-19,Movie,1,0,0,0,0,0,...,0,0,0,0,0,panslabyrinth,https://occ-0-2042-299.1.nflxso.net/dnm/api/v6...,판의 미로 - 오필리아와 세 개의 열쇠,panslabyrinth,7478.0
4756,15214,The Twilight Saga New Moon,2009-11-20,Movie,1,0,0,0,1,0,...,0,0,0,0,0,thetwilightsaganewmoon,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,뉴 문,thetwilightsaganewmoon,9588.0
4800,15358,Battle Los Angeles,2011-03-11,Movie,0,0,1,0,0,0,...,0,0,0,0,0,battlelosangeles,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,월드 인베이젼,battlelosangeles,7410.0


In [21]:
temp_df2.loc[temp_df2.series_or_movies == 'Series', 'num_of_netizen'] = None
temp_df2.loc[temp_df2.index == 4040, 'num_of_netizen'] = None

In [22]:
temp_df2[temp_df2.duplicated(subset=['title'], keep=False)]

,Unnamed: 0,title,release_date,series_or_movies,drama,comedy,action,thriller,romance,crime,...,musical,reality-tv,western,game-show,talk-show,Title for Crawling,Image,korean_title,english_title,num_of_netizen
293,1278,Start-Up,2019-12-18,Series,1,1,1,0,0,0,...,0,0,0,0,0,startup,https://occ-0-1091-299.1.nflxso.net/dnm/api/v6...,시동,startup,NaN
501,1948,About Time,2013-11-08,Series,1,1,0,0,1,0,...,0,0,0,0,0,abouttime,https://occ-0-58-2773.1.nflxso.net/dnm/api/v6/...,어바웃 타임,abouttime,NaN
768,3258,Snowpiercer,2014-07-11,Series,1,0,1,0,0,0,...,0,0,0,0,0,snowpiercer,https://occ-0-1091-299.1.nflxso.net/dnm/api/v6...,설국열차,snowpiercer,NaN
901,3708,Start-Up,2019-12-18,Movie,1,1,1,0,0,0,...,0,0,0,0,0,startup,https://occ-0-1007-1360.1.nflxso.net/dnm/api/v...,시동,startup,10726.0
1427,5231,Be with You,2018-04-06,Movie,1,0,0,0,1,0,...,0,0,0,0,0,bewithyou,https://occ-0-1007-1360.1.nflxso.net/dnm/api/v...,지금 만나러 갑니다,bewithyou,13854.0
3657,10933,Be with You,2015-10-31,Series,0,0,0,0,0,0,...,0,0,0,0,0,bewithyou,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,지금 만나러 갑니다,bewithyou,NaN
4040,12142,Be with You,2004-10-30,Movie,1,0,0,0,1,0,...,0,0,0,0,0,bewithyou,https://occ-0-1007-1360.1.nflxso.net/dnm/api/v...,지금 만나러 갑니다,bewithyou,NaN
4106,12410,About Time,2013-11-08,Movie,1,1,0,0,1,0,...,0,0,0,0,0,abouttime,https://occ-0-2219-2218.1.nflxso.net/dnm/api/v...,어바웃 타임,abouttime,15009.0
4450,13920,Snowpiercer,2014-07-11,Movie,1,0,1,0,0,0,...,0,0,0,0,0,snowpiercer,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,설국열차,snowpiercer,37844.0


In [23]:
target_movie = temp_df2[temp_df2.duplicated(subset=['title'], keep=False)]
target_movie[target_movie['num_of_netizen'].isna()].index

Int64Index([293, 501, 768, 3657, 4040], dtype='int64')

In [24]:
temp_df2 = temp_df2.drop(target_movie[target_movie['num_of_netizen'].isna()].index)

In [25]:
temp_df2 = temp_df2.rename(columns={'Unnamed: 0':'old_index'})
temp_df2

,old_index,title,release_date,series_or_movies,drama,comedy,action,thriller,romance,crime,...,musical,reality-tv,western,game-show,talk-show,Title for Crawling,Image,korean_title,english_title,num_of_netizen
7,58,Miss Baek,2018-10-11,Movie,1,0,0,0,0,0,...,0,0,0,0,0,missbaek,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,미쓰백,missbaek,8671.0
23,74,Keys To The Heart,2018-01-26,Movie,1,1,0,0,0,0,...,0,0,0,0,0,keystotheheart,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,그것만이 내 세상,keystotheheart,15829.0
24,75,Intimate Strangers,2004-09-03,Movie,1,0,0,1,1,0,...,0,0,0,0,0,intimatestrangers,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,완벽한 타인,intimatestrangers,23874.0
27,78,Gonjiam Haunted Asylum,2018-03-28,Movie,0,0,0,0,0,0,...,0,0,0,0,0,gonjiamhauntedasylum,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,곤지암,gonjiamhauntedasylum,16845.0
28,79,Golden Slumber,2010-01-30,Movie,1,0,1,1,0,0,...,0,0,0,0,0,goldenslumber,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,골든슬럼버,goldenslumber,9609.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,14747,The Bourne Ultimatum,2007-08-03,Movie,0,0,1,1,0,0,...,0,0,0,0,0,thebourneultimatum,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,본 얼티메이텀,thebourneultimatum,8715.0
4738,15141,Pans Labyrinth,2007-01-19,Movie,1,0,0,0,0,0,...,0,0,0,0,0,panslabyrinth,https://occ-0-2042-299.1.nflxso.net/dnm/api/v6...,판의 미로 - 오필리아와 세 개의 열쇠,panslabyrinth,7478.0
4756,15214,The Twilight Saga New Moon,2009-11-20,Movie,1,0,0,0,1,0,...,0,0,0,0,0,thetwilightsaganewmoon,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,뉴 문,thetwilightsaganewmoon,9588.0
4800,15358,Battle Los Angeles,2011-03-11,Movie,0,0,1,0,0,0,...,0,0,0,0,0,battlelosangeles,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,월드 인베이젼,battlelosangeles,7410.0


In [32]:
temp_df2.iloc[1,-4]

'https://occ-0-1361-988.1.nflxso.net/dnm/api/v6/evlCitJPPCVCry0BZlEFb5-QjKc/AAAABRbFw-Te0EQ4Xuva03luk_sW4C3tBxJpqZST2J5pIVjOGAYbyNg4GY7DO90-n3lTHXnUVn-eJr99zr-fnkSYEPyt9A.jpg?r=25c'

In [33]:
temp_df2.iloc[100,-4]

'https://occ-0-1007-1360.1.nflxso.net/dnm/api/v6/evlCitJPPCVCry0BZlEFb5-QjKc/AAAAAU9QJKYyW5QPmBzXBy8WKZo50F9MDe8FH4CI_bN1WFAu2IUFgDUKhOix6-lvXzjkARKpTc7fn4WDDS4_O3aG3H8-3g.jpg?r=385'

In [27]:
temp_df2.sum()

old_index                                                       1711402
title                 Miss BaekKeys To The HeartIntimate StrangersGo...
series_or_movies      MovieMovieMovieMovieMovieMovieMovieMovieMovieM...
drama                                                               101
comedy                                                               46
action                                                               99
thriller                                                             66
romance                                                              29
crime                                                                49
adventure                                                            45
animation                                                             7
fantasy                                                              25
family                                                                9
sci-fi                                                          

In [28]:
temp_df2[temp_df2['drama'] == 1]

,old_index,title,release_date,series_or_movies,drama,comedy,action,thriller,romance,crime,...,musical,reality-tv,western,game-show,talk-show,Title for Crawling,Image,korean_title,english_title,num_of_netizen
7,58,Miss Baek,2018-10-11,Movie,1,0,0,0,0,0,...,0,0,0,0,0,missbaek,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,미쓰백,missbaek,8671.0
23,74,Keys To The Heart,2018-01-26,Movie,1,1,0,0,0,0,...,0,0,0,0,0,keystotheheart,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,그것만이 내 세상,keystotheheart,15829.0
24,75,Intimate Strangers,2004-09-03,Movie,1,0,0,1,1,0,...,0,0,0,0,0,intimatestrangers,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,완벽한 타인,intimatestrangers,23874.0
28,79,Golden Slumber,2010-01-30,Movie,1,0,1,1,0,0,...,0,0,0,0,0,goldenslumber,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,골든슬럼버,goldenslumber,9609.0
30,81,Default,2018-11-28,Movie,1,0,0,0,0,0,...,0,0,0,0,0,default,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,국가부도의 날,default,14744.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4549,14478,The Dark Knight,2008-07-18,Movie,1,0,1,1,0,1,...,0,0,0,0,0,thedarkknight,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,다크 나이트,thedarkknight,27302.0
4600,14685,Forrest Gump,1994-07-06,Movie,1,0,0,0,1,0,...,0,0,0,0,0,forrestgump,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,포레스트 검프,forrestgump,8387.0
4738,15141,Pans Labyrinth,2007-01-19,Movie,1,0,0,0,0,0,...,0,0,0,0,0,panslabyrinth,https://occ-0-2042-299.1.nflxso.net/dnm/api/v6...,판의 미로 - 오필리아와 세 개의 열쇠,panslabyrinth,7478.0
4756,15214,The Twilight Saga New Moon,2009-11-20,Movie,1,0,0,0,1,0,...,0,0,0,0,0,thetwilightsaganewmoon,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,뉴 문,thetwilightsaganewmoon,9588.0


- 여기서 앗싸리 500개 받아온 다음 각 장르별 숫자 균형 맞게끔 조정하기

In [35]:
# temp_df2.to_csv('../data/favorite_contents_netflix_movie.csv', sep=',', encoding='utf-8-sig')

#  favorite top 200 netflix movie data

In [47]:
favorite_top200_df

,old_index,title,release_date,series_or_movies,drama,comedy,action,thriller,romance,crime,...,musical,reality-tv,western,game-show,talk-show,Title for Crawling,Image,korean_title,english_title,num_of_netizen
0,58,Miss Baek,2018-10-11,Movie,1,0,0,0,0,0,...,0,0,0,0,0,missbaek,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,미쓰백,missbaek,8671
1,74,Keys To The Heart,2018-01-26,Movie,1,1,0,0,0,0,...,0,0,0,0,0,keystotheheart,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,그것만이 내 세상,keystotheheart,15829
2,75,Intimate Strangers,2004-09-03,Movie,1,0,0,1,1,0,...,0,0,0,0,0,intimatestrangers,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,완벽한 타인,intimatestrangers,23874
3,78,Gonjiam Haunted Asylum,2018-03-28,Movie,0,0,0,0,0,0,...,0,0,0,0,0,gonjiamhauntedasylum,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,곤지암,gonjiamhauntedasylum,16845
4,79,Golden Slumber,2010-01-30,Movie,1,0,1,1,0,0,...,0,0,0,0,0,goldenslumber,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,골든슬럼버,goldenslumber,9609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,14747,The Bourne Ultimatum,2007-08-03,Movie,0,0,1,1,0,0,...,0,0,0,0,0,thebourneultimatum,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,본 얼티메이텀,thebourneultimatum,8715
196,15141,Pans Labyrinth,2007-01-19,Movie,1,0,0,0,0,0,...,0,0,0,0,0,panslabyrinth,https://occ-0-2042-299.1.nflxso.net/dnm/api/v6...,판의 미로 - 오필리아와 세 개의 열쇠,panslabyrinth,7478
197,15214,The Twilight Saga New Moon,2009-11-20,Movie,1,0,0,0,1,0,...,0,0,0,0,0,thetwilightsaganewmoon,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,뉴 문,thetwilightsaganewmoon,9588
198,15358,Battle Los Angeles,2011-03-11,Movie,0,0,1,0,0,0,...,0,0,0,0,0,battlelosangeles,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,월드 인베이젼,battlelosangeles,7410


# column renamed neflix_df

In [52]:
to_rename = {'Series or Movies':'series_or_movies', 'Release Date':'release_date', 'Title':'title', 'Unnamed: 0':'old_index'}
netflix_df = netflix_df.rename(columns=to_rename)
netflix_df

,old_index,title,release_date,series_or_movies,drama,comedy,action,thriller,romance,crime,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,Title for Crawling,Image
0,4,Moxie,2011-09-22,Movie,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,moxie,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...
1,27,Comrades Almost a Love Story,1996-11-02,Movie,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,comradesalmostalovestory,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...
2,28,The Closet,2001-08-10,Movie,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,thecloset,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...
3,53,The Bold Type,2017-06-20,Series,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,theboldtype,https://occ-0-768-769.1.nflxso.net/dnm/api/v6/...
4,55,Lobster Cop,2018-06-22,Movie,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,lobstercop,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4840,15475,K-POP Extreme Survival,NaN,Series,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,kpopextremesurvival,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...
4841,15476,DreamWorks Shreks Swamp Stories,NaN,Series,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,dreamworksshreksswampstories,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...
4842,15477,DreamWorks Happy Holidays from Madagascar,NaN,Series,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,dreamworkshappyholidaysfrommadagascar,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...
4843,15478,DreamWorks Holiday Classics,NaN,Series,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,dreamworksholidayclassics,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...


# 15000 neflix data 

- preprocessing genre to list then one hot encoding
- picking index, preprossed genre, img_link, imdb vots

In [54]:
total_df.head()

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,Writer,...,Netflix Release Date,Production House,Netflix Link,IMDb Link,Summary,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,...,2021-03-04,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN
1,HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Movie,7.0,Canada,1-2 hour,Coky Giedroyc,Caitlin Moran,...,2021-03-04,"Film 4, Monumental Pictures, Lionsgate",https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,YouTube
2,Centigrade,"Drama, Thriller",Thrillers,English,Movie,6.4,Canada,1-2 hour,Brendan Walsh,"Brendan Walsh, Daley Nixon",...,2021-03-04,NaN,https://www.netflix.com/watch/81305978,https://www.imdb.com/title/tt8945942,"Trapped in a frozen car during a blizzard, a p...",1720.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BODM2MD...,https://www.youtube.com/watch?v=0RvV7TNUlkQ,YouTube
3,ANNE+,Drama,"TV Dramas,Romantic TV Dramas,Dutch TV Shows",Turkish,Series,7.7,"Belgium,Netherlands",< 30 minutes,NaN,NaN,...,2021-03-04,NaN,https://www.netflix.com/watch/81336456,https://www.imdb.com/title/tt6132758,"Upon moving into a new place, a 20-something r...",1147.0,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BNWRkMz...,NaN,NaN
4,Moxie,"Animation, Short, Drama","Social Issue Dramas,Teen Movies,Dramas,Comedie...",English,Movie,8.1,"Lithuania,Poland,France,Iceland,Italy,Spain,Gr...",1-2 hour,Stephen Irwin,NaN,...,2021-03-04,NaN,https://www.netflix.com/watch/81078393,https://www.imdb.com/title/tt2023611,Inspired by her moms rebellious past and a con...,63.0,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BODYyNW...,NaN,NaN


### genre preprocessing 

In [55]:
genre_before_rg = total_df['Genre']
reg_genre = []

for i in range(len(genre_before_rg)):
    try:
        target = total_df.iloc[i,1].replace(' ','').lower().split(',')
        reg_genre.append(target)

#         print(target)
    except:
        reg_genre.append([])

In [56]:
total_df_genre_lst_df = total_df.copy()

In [57]:
total_df_genre_lst_df.insert(2, 'genre_list', reg_genre)

In [62]:
print(type(total_df_genre_lst_df['genre_list'][0]), total_df_genre_lst_df['genre_list'][0])
total_df_genre_lst_df

<class 'list'> ['crime', 'drama', 'fantasy', 'horror', 'romance']


,Title,Genre,genre_list,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,...,Netflix Release Date,Production House,Netflix Link,IMDb Link,Summary,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","[crime, drama, fantasy, horror, romance]","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,...,2021-03-04,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN
1,HOW TO BUILD A GIRL,Comedy,[comedy],"Dramas,Comedies,Films Based on Books,British",English,Movie,7.0,Canada,1-2 hour,Coky Giedroyc,...,2021-03-04,"Film 4, Monumental Pictures, Lionsgate",https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,YouTube
2,Centigrade,"Drama, Thriller","[drama, thriller]",Thrillers,English,Movie,6.4,Canada,1-2 hour,Brendan Walsh,...,2021-03-04,NaN,https://www.netflix.com/watch/81305978,https://www.imdb.com/title/tt8945942,"Trapped in a frozen car during a blizzard, a p...",1720.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BODM2MD...,https://www.youtube.com/watch?v=0RvV7TNUlkQ,YouTube
3,ANNE+,Drama,[drama],"TV Dramas,Romantic TV Dramas,Dutch TV Shows",Turkish,Series,7.7,"Belgium,Netherlands",< 30 minutes,NaN,...,2021-03-04,NaN,https://www.netflix.com/watch/81336456,https://www.imdb.com/title/tt6132758,"Upon moving into a new place, a 20-something r...",1147.0,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BNWRkMz...,NaN,NaN
4,Moxie,"Animation, Short, Drama","[animation, short, drama]","Social Issue Dramas,Teen Movies,Dramas,Comedie...",English,Movie,8.1,"Lithuania,Poland,France,Iceland,Italy,Spain,Gr...",1-2 hour,Stephen Irwin,...,2021-03-04,NaN,https://www.netflix.com/watch/81078393,https://www.imdb.com/title/tt2023611,Inspired by her moms rebellious past and a con...,63.0,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BODYyNW...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15475,K-POP Extreme Survival,NaN,[],"TV Dramas,TV Programmes,TV Comedies,Romantic T...",NaN,Series,NaN,"South Korea,Argentina,United Kingdom,Australia...",< 30 minutes,NaN,...,2015-04-14,NaN,https://www.netflix.com/watch/80031595,NaN,Seung Yeon decides to chase her dream of becom...,NaN,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN
15476,DreamWorks Shreks Swamp Stories,NaN,[],"Animal Tales,Family Comedies,Family Adventures...",NaN,Series,NaN,"Russia,Hong Kong,Hungary,Australia,South Korea...",< 30 minutes,NaN,...,2015-04-14,NaN,https://www.netflix.com/watch/70242630,NaN,"Shrek celebrates Halloween, Puss in Boots is c...",NaN,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN
15477,DreamWorks Happy Holidays from Madagascar,"Animation, Comedy, Family","[animation, comedy, family]","TV Comedies,Kids TV,Animal Tales,TV Cartoons,T...",English,Series,8.4,"Belgium,Switzerland,United States,Germany,Unit...",< 30 minutes,NaN,...,2015-04-14,NaN,https://www.netflix.com/watch/70259784,NaN,Madagascar goes wild with holiday spirit in th...,71.0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,NaN,NaN,NaN
15478,DreamWorks Holiday Classics,"Animation, Comedy, Family","[animation, comedy, family]","TV Comedies,Kids TV,TV Cartoons,TV Programmes,...",English,Series,8.2,"Belgium,Switzerland,Germany,Argentina,United K...",< 30 minutes,NaN,...,2015-04-14,Fox,https://www.netflix.com/watch/70221348,

###  genre OHE

In [63]:
def to_1D(series):
 return pd.Series([x for _list in series for x in _list])

In [64]:
values = to_1D(total_df_genre_lst_df['genre_list']).value_counts().keys().tolist()
counts = to_1D(total_df_genre_lst_df['genre_list']).value_counts().tolist()
unique_items = dict(zip(values, counts))
unique_items

{'drama': 6359,
 'comedy': 5077,
 'action': 2810,
 'thriller': 2739,
 'romance': 2445,
 'crime': 1932,
 'adventure': 1809,
 'animation': 1665,
 'fantasy': 1594,
 'family': 1433,
 'sci-fi': 1204,
 'mystery': 1190,
 'horror': 1070,
 'documentary': 1030,
 'biography': 636,
 'history': 527,
 'music': 426,
 'short': 422,
 'sport': 367,
 'war': 330,
 'musical': 228,
 'reality-tv': 191,
 'western': 109,
 'game-show': 52,
 'talk-show': 26,
 'news': 20,
 'adult': 15,
 'film-noir': 2}

In [65]:
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

In [66]:
genre_bool_df = boolean_df(total_df_genre_lst_df['genre_list'], unique_items)

In [67]:
genre_bool_df.head()

,drama,comedy,action,thriller,romance,crime,adventure,animation,fantasy,family,...,sport,war,musical,reality-tv,western,game-show,talk-show,news,adult,film-noir
0,True,False,False,False,True,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [68]:
genre_int_df = genre_bool_df.astype(int)

In [69]:
genre_int_df.head()

,drama,comedy,action,thriller,romance,crime,adventure,animation,fantasy,family,...,sport,war,musical,reality-tv,western,game-show,talk-show,news,adult,film-noir
0,1,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
total_df_genre_lst_df.head(1)

,Title,Genre,genre_list,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,...,Netflix Release Date,Production House,Netflix Link,IMDb Link,Summary,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","[crime, drama, fantasy, horror, romance]","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,...,2021-03-04,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN


In [71]:
favorite_neflix_df = pd.merge(total_df_genre_lst_df[['genre_list', 'Image', 'IMDb Votes']], genre_int_df, left_index=True, right_index=True)

In [77]:
favorite_neflix_df = favorite_neflix_df.drop(columns=favorite_neflix_df.columns[-3:])

In [79]:
favorite_neflix_df.head()

,genre_list,Image,IMDb Votes,drama,comedy,action,thriller,romance,crime,adventure,...,history,music,short,sport,war,musical,reality-tv,western,game-show,talk-show
0,"[crime, drama, fantasy, horror, romance]",https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,205926.0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,[comedy],https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,2838.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[drama, thriller]",https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,1720.0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,[drama],https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,1147.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[animation, short, drama]",https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,63.0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


###  korea netflix data

In [74]:
netflix_df.head()

,old_index,title,release_date,series_or_movies,drama,comedy,action,thriller,romance,crime,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,Title for Crawling,Image
0,4,Moxie,2011-09-22,Movie,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,moxie,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...
1,27,Comrades Almost a Love Story,1996-11-02,Movie,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,comradesalmostalovestory,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...
2,28,The Closet,2001-08-10,Movie,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,thecloset,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...
3,53,The Bold Type,2017-06-20,Series,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,theboldtype,https://occ-0-768-769.1.nflxso.net/dnm/api/v6/...
4,55,Lobster Cop,2018-06-22,Movie,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,lobstercop,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...


## Merge  favorite_neflix_df and netflix_df

In [80]:
favorite_neflix_df.head(1)

,genre_list,Image,IMDb Votes,drama,comedy,action,thriller,romance,crime,adventure,...,history,music,short,sport,war,musical,reality-tv,western,game-show,talk-show
0,"[crime, drama, fantasy, horror, romance]",https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,205926.0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
favorite_neflix_df.columns

Index(['genre_list', 'Image', 'IMDb Votes', 'drama', 'comedy', 'action',
       'thriller', 'romance', 'crime', 'adventure', 'animation', 'fantasy',
       'family', 'sci-fi', 'mystery', 'horror', 'documentary', 'biography',
       'history', 'music', 'short', 'sport', 'war', 'musical', 'reality-tv',
       'western', 'game-show', 'talk-show'],
      dtype='object')

In [83]:
changed_order = ['genre_list', 'drama', 'comedy', 'action',
       'thriller', 'romance', 'crime', 'adventure', 'animation', 'fantasy',
       'family', 'sci-fi', 'mystery', 'horror', 'documentary', 'biography',
       'history', 'music', 'short', 'sport', 'war', 'musical', 'reality-tv',
       'western', 'game-show', 'talk-show', 'Image', 'IMDb Votes']

In [85]:
favorite_neflix_df = favorite_neflix_df[changed_order]
favorite_neflix_df.head(1)

,genre_list,drama,comedy,action,thriller,romance,crime,adventure,animation,fantasy,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,Image,IMDb Votes
0,"[crime, drama, fantasy, horror, romance]",1,0,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,205926.0


In [87]:
merged_fv_nflx_df = pd.merge(netflix_df[['old_index', 'title', 'release_date', 'series_or_movies']], 
                             favorite_neflix_df, how='inner', left_on='old_index', right_index=True)

In [118]:
merged_fv_nflx_df = merged_fv_nflx_df.rename(columns={'Image':'img_link', 'IMDb Votes':'imdb_votes'})
print(merged_fv_nflx_df.shape)
merged_fv_nflx_df.head()

(4845, 32)


,old_index,title,release_date,series_or_movies,genre_list,drama,comedy,action,thriller,romance,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,img_link,imdb_votes
0,4,Moxie,2011-09-22,Movie,"[애니메이션, 단편, 드라마]",1,0,0,0,0,...,1,0,0,0,0,0,0,0,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,63.0
1,27,Comrades Almost a Love Story,1996-11-02,Movie,"[드라마, 로맨스]",1,0,0,0,1,...,0,0,0,0,0,0,0,0,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...,5047.0
2,28,The Closet,2001-08-10,Movie,[코미디],0,1,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...,16657.0
3,53,The Bold Type,2017-06-20,Series,[드라마],1,0,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-768-769.1.nflxso.net/dnm/api/v6/...,9767.0
4,55,Lobster Cop,2018-06-22,Movie,"[코미디, 범죄]",0,1,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,143.0


In [119]:
merged_fv_nflx_df.sum()

old_index                                                    37557751
title               MoxieComrades  Almost a Love StoryThe ClosetTh...
series_or_movies    MovieMovieMovieSeriesMovieMovieMovieMovieMovie...
genre_list          [애니메이션, 단편, 드라마, 드라마, 로맨스, 코미디, 드라마, 코미디, 범죄, ...
drama                                                            2020
comedy                                                           1610
action                                                            923
thriller                                                          863
romance                                                           742
crime                                                             639
adventure                                                         618
animation                                                         622
fantasy                                                           573
family                                                            461
sci-fi              

###  Translate genre_list to Korean

In [120]:
merged_fv_nflx_df.columns

Index(['old_index', 'title', 'release_date', 'series_or_movies', 'genre_list',
       'drama', 'comedy', 'action', 'thriller', 'romance', 'crime',
       'adventure', 'animation', 'fantasy', 'family', 'sci-fi', 'mystery',
       'horror', 'documentary', 'biography', 'history', 'music', 'short',
       'sport', 'war', 'musical', 'reality-tv', 'western', 'game-show',
       'talk-show', 'img_link', 'imdb_votes'],
      dtype='object')

In [121]:
to_translate_dict = {
    'drama':'드라마', 'comedy':'코미디', 'action':'액션', 'thriller':'스릴러', 'romance':'로맨스', 'crime':'범죄',
       'adventure':'모험', 'animation':'애니메이션', 'fantasy':'판타지', 'family':'가족', 'sci-fi':'SF', 'mystery':'미스터리',
       'horror':'공포', 'documentary':'다큐멘터리', 'biography':'전기', 'history':'역사', 'music':'음악', 'short':'단편',
       'sport':'스포츠', 'war':'전쟁', 'musical':'뮤지컬', 'reality-tv':'리얼리티', 'western':'서부', 'game-show':'게임쇼',
       'talk-show':'토크쇼'
}

In [459]:
genre_list = merged_fv_nflx_df.iloc[0,4]
genre_list

['애니메이션', '단편', '드라마']

In [460]:
[re.sub('|'.join(to_translate_dict.keys()), lambda x: to_translate_dict[x.group()], i) for i in genre_list]

['애니메이션', '단편', '드라마']

- http://www.leejungmin.org/post/2018/04/21/pandas_apply_and_map/
-https://stackoverflow.com/questions/35314373/replace-strings-in-list-from-dictionary

In [124]:
def genre_translator(genre_list):
    return [re.sub('|'.join(to_translate_dict.keys()), lambda x: to_translate_dict[x.group()], i) for i in genre_list]

In [125]:
merged_fv_nflx_df['genre_list'] = merged_fv_nflx_df['genre_list'].map(lambda x: genre_translator(x))

In [126]:
print(merged_fv_nflx_df.shape)
merged_fv_nflx_df.head()

(4845, 32)


,old_index,title,release_date,series_or_movies,genre_list,drama,comedy,action,thriller,romance,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,img_link,imdb_votes
0,4,Moxie,2011-09-22,Movie,"[애니메이션, 단편, 드라마]",1,0,0,0,0,...,1,0,0,0,0,0,0,0,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,63.0
1,27,Comrades Almost a Love Story,1996-11-02,Movie,"[드라마, 로맨스]",1,0,0,0,1,...,0,0,0,0,0,0,0,0,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...,5047.0
2,28,The Closet,2001-08-10,Movie,[코미디],0,1,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-395-988.1.nflxso.net/dnm/api/v6/...,16657.0
3,53,The Bold Type,2017-06-20,Series,[드라마],1,0,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-768-769.1.nflxso.net/dnm/api/v6/...,9767.0
4,55,Lobster Cop,2018-06-22,Movie,"[코미디, 범죄]",0,1,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-1361-988.1.nflxso.net/dnm/api/v6...,143.0


##  sort_value by imdb_votes if series_or_movies == series

In [624]:
most_popular_series_df = merged_fv_nflx_df[merged_fv_nflx_df['series_or_movies']=='Series'].sort_values(by='imdb_votes', ascending=False).head(500)
most_popular_movies_df = merged_fv_nflx_df[merged_fv_nflx_df['series_or_movies']=='Movie'].sort_values(by='imdb_votes', ascending=False).head(500)
# most_popular_series_df = merged_fv_nflx_df.sort_values(by='imdb_votes', ascending=False)
most_popular_contents_df = pd.concat([most_popular_series_df, most_popular_movies_df], axis=0)
most_popular_contents_df = most_popular_contents_df.sort_values(by='imdb_votes', ascending=False)
most_popular_contents_df

,old_index,title,release_date,series_or_movies,genre_list,drama,comedy,action,thriller,romance,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,img_link,imdb_votes
4549,14478,The Dark Knight,2008-07-18,Movie,"[액션, 범죄, 드라마, 스릴러]",1,0,1,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,2308981.0
4520,14331,Inception,2010-07-16,Movie,"[액션, 모험, SF, 스릴러]",0,0,1,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,2072912.0
4600,14685,Forrest Gump,1994-07-06,Movie,"[드라마, 로맨스]",1,0,0,0,1,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,1814709.0
4326,13360,The Dark Knight Rises,2012-07-20,Movie,"[액션, 모험]",0,0,1,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,1522306.0
4014,12034,Interstellar,2014-11-07,Movie,"[모험, 드라마, SF]",1,0,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,1516941.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2880,9023,My Next Guest Needs No Introduction With David...,2018-01-12,Series,[토크쇼],0,0,0,0,0,...,0,0,0,0,0,0,0,1,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,4761.0
4751,15199,Winx Club,2004-06-19,Series,"[애니메이션, 액션, 모험, 코미디, 가족, 판타지, 스릴러]",0,1,1,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,4701.0
4241,12936,Kakegurui,2018-02-01,Series,"[애니메이션, 드라마, 미스터리, 스릴러]",1,0,0,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,4701.0
2153,6998,Goblin Slayer,2018-10-01,Series,"[애니메이션, 액션, 모험, 판타지]",0,0,1,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,4700.0


#### select contents by genre

In [625]:
most_popular_contents_df.columns

Index(['old_index', 'title', 'release_date', 'series_or_movies', 'genre_list',
       'drama', 'comedy', 'action', 'thriller', 'romance', 'crime',
       'adventure', 'animation', 'fantasy', 'family', 'sci-fi', 'mystery',
       'horror', 'documentary', 'biography', 'history', 'music', 'short',
       'sport', 'war', 'musical', 'reality-tv', 'western', 'game-show',
       'talk-show', 'img_link', 'imdb_votes'],
      dtype='object')

In [626]:
genre_list = ['drama', 'comedy', 'action', 'thriller', 'romance', 'crime',
       'adventure', 'animation', 'fantasy', 'family', 'sci-fi', 'mystery',
       'horror', 'documentary', 'biography', 'history', 'music', 'short',
       'sport', 'war', 'musical', 'reality-tv', 'western', 'game-show',
       'talk-show']

In [627]:
yes = most_popular_contents_df[most_popular_contents_df['drama'] == 1].head(30).index.values.astype(list)
print(*yes)

4549 4600 4014 4821 4536 4625 4505 4575 4531 4630 4453 4501 256 4595 4745 4609 3753 4664 3788 4486 3698 3623 4098 4548 4542 4508 4738 4545 3959 4066


In [628]:
index_collection_list = []
for i in genre_list:
    try:
        indice = most_popular_contents_df[most_popular_contents_df[i] == 1].head(50).index.values.astype(list)
        for j in indice:
            index_collection_list.append(j)
    except:
        print('no')

In [629]:
index_collection_list

[4549,
 4600,
 4014,
 4821,
 4536,
 4625,
 4505,
 4575,
 4531,
 4630,
 4453,
 4501,
 256,
 4595,
 4745,
 4609,
 3753,
 4664,
 3788,
 4486,
 3698,
 3623,
 4098,
 4548,
 4542,
 4508,
 4738,
 4545,
 3959,
 4066,
 4027,
 4773,
 3638,
 4364,
 4166,
 4538,
 4111,
 4620,
 4109,
 4823,
 2225,
 464,
 4703,
 4705,
 4457,
 2233,
 4655,
 3782,
 4757,
 4769,
 4616,
 4501,
 3648,
 4728,
 4552,
 4360,
 4548,
 4611,
 3960,
 4066,
 4569,
 4690,
 4325,
 2710,
 4607,
 4647,
 464,
 2580,
 4807,
 4597,
 4563,
 4642,
 4518,
 4012,
 4613,
 3840,
 2546,
 4115,
 2711,
 4579,
 4571,
 4803,
 4599,
 4512,
 3931,
 4748,
 4835,
 4134,
 2215,
 4626,
 4476,
 4516,
 501,
 4106,
 4338,
 4567,
 4495,
 3894,
 4643,
 4777,
 4549,
 4520,
 4326,
 4551,
 3648,
 4581,
 4714,
 3849,
 389,
 3946,
 765,
 1163,
 3698,
 3873,
 4030,
 4320,
 4685,
 4606,
 2548,
 3457,
 4753,
 4622,
 3267,
 4399,
 451,
 4110,
 4598,
 3960,
 4521,
 3528,
 4365,
 4318,
 2547,
 4325,
 4773,
 4621,
 4369,
 3638,
 4364,
 4507,
 4538,
 4620,
 4319,
 2225,

In [630]:
print(len(index_collection_list))
print(len(set(index_collection_list)))

903
461


In [631]:
selected_indice_list = list(set(index_collection_list))

####  make genre counts even

In [632]:
select_genre_df = most_popular_contents_df.loc[selected_indice_list]
select_genre_df

,old_index,title,release_date,series_or_movies,genre_list,drama,comedy,action,thriller,romance,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,img_link,imdb_votes
4098,12381,The Imitation Game,2014-12-25,Movie,"[전기, 드라마, 스릴러, 전쟁]",1,0,0,1,0,...,0,0,1,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,686497.0
4103,12403,Dawn of the Planet of the Apes,2014-07-11,Movie,"[액션, 모험, 드라마, SF]",1,0,1,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,412476.0
4106,12410,About Time,2013-11-08,Movie,"[코미디, 드라마, 판타지, 로맨스, SF]",1,1,0,0,1,...,0,0,0,0,0,0,0,0,https://occ-0-2219-2218.1.nflxso.net/dnm/api/v...,304576.0
4108,12418,Sherlock Holmes A Game of Shadows,2011-12-16,Movie,"[액션, 모험, 미스터리]",0,0,1,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,421586.0
4109,12420,Zodiac,2007-03-02,Movie,"[범죄, 드라마, 미스터리, 스릴러]",1,0,0,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,467977.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4067,12232,Gotham,2014-09-22,Series,"[액션, 범죄, 드라마, 미스터리, SF, 스릴러]",1,0,1,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,190942.0
2030,6694,Aliens vs. Predator 2,2007-12-25,Movie,"[액션, 모험, 판타지, 공포, SF, 스릴러]",0,0,1,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-64-325.1.nflxso.net/dnm/api/v6/e...,116362.0
4080,12276,Prometheus,2012-06-08,Movie,"[모험, 미스터리, SF]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-64-325.1.nflxso.net/dnm/api/v6/e...,569426.0
4083,12289,The Maze Runner,2014-09-19,Movie,"[액션, 미스터리, SF, 스릴러]",0,0,1,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,418958.0


In [633]:
select_genre_df.groupby('series_or_movies').count()

,old_index,title,release_date,genre_list,drama,comedy,action,thriller,romance,crime,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,img_link,imdb_votes
series_or_movies,,,,,,,,,,,,,,,,,,,,,
Movie,322,322,322,322,322,322,322,322,322,322,...,322,322,322,322,322,322,322,322,322,322
Series,139,139,138,139,139,139,139,139,139,139,...,139,139,139,139,139,139,139,139,139,139


In [634]:
select_genre_df.sum()

old_index                                                     5070499
title               The Imitation GameDawn of the Planet of the Ap...
series_or_movies    MovieMovieMovieMovieMovieMovieMovieMovieMovieM...
genre_list          [전기, 드라마, 스릴러, 전쟁, 액션, 모험, 드라마, SF, 코미디, 드라마, ...
drama                                                             227
comedy                                                            125
action                                                            141
thriller                                                          122
romance                                                            74
crime                                                              91
adventure                                                         146
animation                                                          60
fantasy                                                            98
family                                                             61
sci-fi              

#### 여전히 드라마가 많다..

In [635]:
select_genre_control_df = select_genre_df.copy()
select_genre_control_df.head()

,old_index,title,release_date,series_or_movies,genre_list,drama,comedy,action,thriller,romance,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,img_link,imdb_votes
4098,12381,The Imitation Game,2014-12-25,Movie,"[전기, 드라마, 스릴러, 전쟁]",1,0,0,1,0,...,0,0,1,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,686497.0
4103,12403,Dawn of the Planet of the Apes,2014-07-11,Movie,"[액션, 모험, 드라마, SF]",1,0,1,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,412476.0
4106,12410,About Time,2013-11-08,Movie,"[코미디, 드라마, 판타지, 로맨스, SF]",1,1,0,0,1,...,0,0,0,0,0,0,0,0,https://occ-0-2219-2218.1.nflxso.net/dnm/api/v...,304576.0
4108,12418,Sherlock Holmes A Game of Shadows,2011-12-16,Movie,"[액션, 모험, 미스터리]",0,0,1,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,421586.0
4109,12420,Zodiac,2007-03-02,Movie,"[범죄, 드라마, 미스터리, 스릴러]",1,0,0,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,467977.0


In [636]:
select_genre_control_df.sum()

old_index                                                     5070499
title               The Imitation GameDawn of the Planet of the Ap...
series_or_movies    MovieMovieMovieMovieMovieMovieMovieMovieMovieM...
genre_list          [전기, 드라마, 스릴러, 전쟁, 액션, 모험, 드라마, SF, 코미디, 드라마, ...
drama                                                             227
comedy                                                            125
action                                                            141
thriller                                                          122
romance                                                            74
crime                                                              91
adventure                                                         146
animation                                                          60
fantasy                                                            98
family                                                             61
sci-fi              

###  Dropping

In [790]:
select_genre_control_df[select_genre_control_df['history']==1]

,old_index,title,release_date,series_or_movies,genre_list,drama,comedy,action,thriller,romance,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,img_link,imdb_votes
2948,9201,The Toys That Made Us,2017-12-22,Series,"[다큐멘터리, 코미디, 역사]",0,1,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,4877.0
248,1124,The Liberator,2014-07-24,Series,"[전기, 드라마, 역사]",1,0,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-1432.1.nflxso.net/dnm/api/v...,5038.0
3444,10388,Scarlet Heart Ryeo,2016-08-29,Series,"[드라마, 판타지, 역사, 로맨스]",1,0,0,0,1,...,0,0,0,0,0,0,0,0,https://occ-0-1007-1360.1.nflxso.net/dnm/api/v...,5111.0
1178,4563,Greatest Events of WWII in Colour,2019-11-08,Series,"[다큐멘터리, 역사, 전쟁]",0,0,0,0,0,...,0,0,1,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,5276.0
457,1760,Three Kingdoms,2008-04-03,Series,"[액션, 드라마, 역사, 전쟁]",1,0,1,0,0,...,0,0,1,0,0,0,0,0,https://occ-0-1007-1360.1.nflxso.net/dnm/api/v...,6097.0
431,1688,High Score,2020-08-19,Series,"[다큐멘터리, 역사]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,6117.0
1453,5285,The Last Czars,2019-07-03,Series,"[다큐멘터리, 드라마, 역사, 로맨스]",1,0,0,0,1,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,6309.0
886,3678,Freud,2020-03-23,Series,"[전기, 범죄, 역사, 스릴러]",0,0,0,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,9664.0
2831,8908,The Frankenstein Chronicles,2018-02-20,Series,"[범죄, 드라마, 판타지, 역사, 미스터리]",1,0,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,9977.0
893,3691,The English Game,2020-03-20,Series,"[드라마, 역사, 스포츠]",1,0,0,0,0,...,0,1,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,11626.0


In [799]:
target_genre = 'drama'
drop_how_many = 5

target_indice_list = select_genre_control_df[select_genre_control_df[target_genre]==1].tail(drop_how_many).index.values.astype(list)

index_control_list = []
for i in target_indice_list:
    index_control_list.append(i)
select_genre_control_df = select_genre_control_df.drop_duplicates(subset=['old_index'])
select_genre_control_df = select_genre_control_df.drop(index_control_list)

In [800]:
print(select_genre_control_df['title'].count())
print(select_genre_control_df.sum())
select_genre_control_df[genre_columns_list].sum()/select_genre_control_df[genre_columns_list].sum().sum()*100

129
old_index                                                     1167847
title               My Next Guest Needs No Introduction With David...
series_or_movies    SeriesSeriesSeriesSeriesSeriesSeriesSeriesSeri...
genre_list          [토크쇼, 리얼리티, 로맨스, 다큐멘터리, 다큐멘터리, 범죄, 다큐멘터리, 코미디,...
drama                                                              32
comedy                                                             31
action                                                             27
thriller                                                           25
romance                                                            13
crime                                                              25
adventure                                                          25
animation                                                          19
fantasy                                                            24
family                                                             19
sci-fi          

drama          7.785888
comedy         7.542579
action         6.569343
thriller       6.082725
romance        3.163017
crime          6.082725
adventure      6.082725
animation      4.622871
fantasy        5.839416
family         4.622871
sci-fi         4.622871
mystery        3.892944
horror         5.839416
documentary    5.839416
biography      2.919708
history        4.136253
music          2.189781
short          1.459854
sport          1.703163
war            1.216545
musical        1.946472
reality-tv     2.676399
western        0.973236
game-show      1.216545
talk-show      0.973236
dtype: float64

###  Adding

In [796]:
genre_to_add = 'biography'
add_how_many = 5
target_indice_list = select_genre_df[select_genre_df[genre_to_add]==1].sort_values(by='imdb_votes', ascending=False).head(add_how_many).index.values.tolist()
select_genre_control_df = pd.concat([select_genre_control_df, select_genre_df.loc[target_indice_list]], axis=0)
select_genre_control_df = select_genre_control_df.drop_duplicates(subset=['old_index'])
select_genre_control_df = select_genre_control_df.sort_values(by='imdb_votes')

In [797]:
print(select_genre_control_df['title'].count())
print(select_genre_control_df.sum())
select_genre_control_df[genre_columns_list].sum()/select_genre_control_df[genre_columns_list].sum().sum()*100

134
old_index                                                     1224270
title               My Next Guest Needs No Introduction With David...
series_or_movies    SeriesSeriesSeriesSeriesSeriesSeriesSeriesSeri...
genre_list          [토크쇼, 리얼리티, 로맨스, 다큐멘터리, 다큐멘터리, 범죄, 다큐멘터리, 코미디,...
drama                                                              37
comedy                                                             31
action                                                             27
thriller                                                           27
romance                                                            13
crime                                                              26
adventure                                                          26
animation                                                          19
fantasy                                                            24
family                                                             19
sci-fi          

drama          8.644860
comedy         7.242991
action         6.308411
thriller       6.308411
romance        3.037383
crime          6.074766
adventure      6.074766
animation      4.439252
fantasy        5.607477
family         4.439252
sci-fi         4.906542
mystery        4.205607
horror         5.607477
documentary    5.607477
biography      3.271028
history        4.205607
music          2.102804
short          1.401869
sport          1.635514
war            1.401869
musical        1.869159
reality-tv     2.570093
western        0.934579
game-show      1.168224
talk-show      0.934579
dtype: float64

In [451]:
select_genre_control_df = select_genre_control_df.drop_duplicates(subset=['old_index'])
select_genre_control_df

,old_index,title,release_date,series_or_movies,genre_list,drama,comedy,action,thriller,romance,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,img_link,imdb_votes
3745,11160,Last Chance U,2016-07-29,Series,"[다큐멘터리, 스포츠]",0,0,0,0,0,...,0,1,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,5981.0
457,1760,Three Kingdoms,2008-04-03,Series,"[액션, 드라마, 역사, 전쟁]",1,0,1,0,0,...,0,0,1,0,0,0,0,0,https://occ-0-1007-1360.1.nflxso.net/dnm/api/v...,6097.0
431,1688,High Score,2020-08-19,Series,"[다큐멘터리, 역사]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,6117.0
1453,5285,The Last Czars,2019-07-03,Series,"[다큐멘터리, 드라마, 역사, 로맨스]",1,0,0,0,1,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,6309.0
1104,4354,The Confession Killer,2019-12-06,Series,"[다큐멘터리, 범죄]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,6395.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,13309,Spider-Man 2,2004-06-30,Movie,"[액션, 모험, SF]",0,0,1,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,545277.0
4365,13528,The Amazing Spider-Man,2012-07-03,Movie,"[액션, 모험, SF]",0,0,1,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,559478.0
4110,12423,Edge of Tomorrow,2014-06-06,Movie,"[액션, 모험, SF]",0,0,1,0,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,601294.0
4477,14086,Memento,2001-05-25,Movie,"[미스터리, 스릴러]",0,0,0,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...,1127675.0


In [461]:
# select_genre_control_df.to_csv('../data/temp_favorite_contents_unified.csv', sep=',', encoding='utf-8-sig')

In [813]:
select_genre_control_df = select_genre_control_df.sort_values(by='imdb_votes',ascending=False)

In [815]:
# select_genre_control_df.to_csv('../data/favorite_contents_unified.csv', sep=',', encoding='utf-8-sig')

## favorite_contents_unified

In [816]:
favorite_contents_unified = pd.read_csv('../data/favorite_contents_unified.csv')

In [819]:
favorite_contents_unified.head(1)

,old_index,title,release_date,series_or_movies,genre_list,drama,comedy,action,thriller,romance,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,img_link,imdb_votes
0,14331,Inception,2010-07-16,Movie,"['액션', '모험', 'SF', '스릴러']",0,0,1,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,2072912


In [808]:
print(favorite_contents_unified.shape)
print(favorite_contents_unified.dtypes)
print(favorite_contents_unified.columns)

(129, 33)
Unnamed: 0            int64
old_index             int64
title                object
release_date         object
series_or_movies     object
genre_list           object
drama                 int64
comedy                int64
action                int64
thriller              int64
romance               int64
crime                 int64
adventure             int64
animation             int64
fantasy               int64
family                int64
sci-fi                int64
mystery               int64
horror                int64
documentary           int64
biography             int64
history               int64
music                 int64
short                 int64
sport                 int64
war                   int64
musical               int64
reality-tv            int64
western               int64
game-show             int64
talk-show             int64
img_link             object
imdb_votes          float64
dtype: object
Index(['Unnamed: 0', 'old_index', 'title', 'release_

In [820]:
favorite_contents_unified.head(1)

,old_index,title,release_date,series_or_movies,genre_list,drama,comedy,action,thriller,romance,...,short,sport,war,musical,reality-tv,western,game-show,talk-show,img_link,imdb_votes
0,14331,Inception,2010-07-16,Movie,"['액션', '모험', 'SF', '스릴러']",0,0,1,1,0,...,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,2072912


In [825]:
essential_df = favorite_contents_unified.drop(columns=['old_index', 'release_date', 'series_or_movies', 'imdb_votes'])
essential_df

,title,genre_list,drama,comedy,action,thriller,romance,crime,adventure,animation,...,music,short,sport,war,musical,reality-tv,western,game-show,talk-show,img_link
0,Inception,"['액션', '모험', 'SF', '스릴러']",0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...
1,Shutter Island,"['미스터리', '스릴러']",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...
2,Memento,"['미스터리', '스릴러']",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...
3,Back to the Future,"['모험', '코미디', 'SF']",0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,https://occ-0-1348-1347.1.nflxso.net/dnm/api/v...
4,Deadpool,"['액션', '모험', '코미디', 'SF']",0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,https://occ-0-2773-2774.1.nflxso.net/dnm/api/v...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,The Toys That Made Us,"['다큐멘터리', '코미디', '역사']",0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...
125,The Confession Tapes,"['다큐멘터리', '범죄']",0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...
126,Abstract The Art of Design,['다큐멘터리'],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...
127,Love Is Blind,"['리얼리티', '로맨스']",0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...


In [824]:
# essential_df.to_csv('../data/favorite_contents_unified_modified.csv', sep=',', encoding='utf-8-sig')

######  genre distribution korea

In [331]:
genre_columns_list = ['drama',
       'comedy', 'action', 'thriller', 'romance', 'crime', 'adventure',
       'animation', 'fantasy', 'family', 'sci-fi', 'mystery', 'horror',
       'documentary', 'biography', 'history', 'music', 'short', 'sport', 'war',
       'musical', 'reality-tv', 'western', 'game-show', 'talk-show']

In [336]:
netflix_df[genre_columns_list].sum().sum()

11886

In [338]:
netflix_df[genre_columns_list].sum()/11886*100

drama          16.994784
comedy         13.545347
action          7.765438
thriller        7.260643
romance         6.242638
crime           5.376073
adventure       5.199394
animation       5.233047
fantasy         4.820798
family          3.878513
sci-fi          3.499916
mystery         3.491503
horror          2.641763
documentary     4.358068
biography       1.665825
history         1.564866
music           1.203096
short           1.455494
sport           1.009591
war             0.757193
musical         0.521622
reality-tv      0.959112
western         0.210331
game-show       0.201918
talk-show       0.143025
dtype: float64

### BE로 보내주기

- title, genre, img_link

   100개 잘라서 
- ver1. title, genre(list), series_or_movies, img_link
- ver2. title, drama, comedy(int), img_link